In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = np.load(r'C:\Users\c\Desktop\KAIST 기초기계학습\all_clf_arr.npz',allow_pickle=True)

In [ ]:
array1 = data['x_arr']
array2 = data['y_arr']
array3 = data['c_arr']
array4 = data['a_arr']

array1.index -> 0:ECG, 1:PPG , 2:CO2

In [ ]:
x = array1
y = array2
c = array3
a = array4

## Model Build

In [ ]:
import os
import time
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score  

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
x = x[~np.isnan(array2)]
y = y[~np.isnan(array2)]
c = c[~np.isnan(array2)]
a = a[~np.isnan(array2)]

print(x.shape)
print(y.shape)
print(c.shape)
print(a.shape)

In [ ]:
print(x.shape)
print(y.shape)
print(c.shape)
print(a.shape)

In [ ]:
x = x[:,:,1].reshape(-1,1,3000)

In [ ]:
print(x.shape)

In [ ]:
# x에서 nonnan,0이상으로만 구성된 데이터 idx추출
idx = []
for i in range(len(x)):
    if (np.isnan(x[i]).sum() == 0)&(np.min(x[i]) > 0):
        idx.append(i)

In [ ]:
#데이터 필터 및 전처리
from pyvital import arr

def process_beat(seg):
    data = arr.interp_undefined(seg.flatten())
    # beat detection
    minlist, maxlist = arr.detect_peaks(data, 100)
    maxlist = maxlist[1:]
    # caseid = caseid.replace(‘.vital’, ‘’)
    # beat lengths
    beatlens = []
    beats = []
    beats_128 = []
    beats_128_valid = []
    for j in range(0, len(minlist)-1):
        beatlen = minlist[j+1] - minlist[j]  # in samps
        beatlens.append(beatlen)
        beat = data[minlist[j]:minlist[j+1]]
        beats.append(beat)
        resampled = arr.resample(beat, 100)  # 임시로 100
        beats_128.append(resampled)
        beats_128_valid.append(resampled)
    avgbeat = np.array(beats_128_valid).mean(axis=0)
    nucase_mbeats = len(maxlist)
    if nucase_mbeats < 10:
        # print(‘nucase_mbeats < 4’)
        return 0, []
    meanlen = np.mean(beatlens)
    stdlen = np.std(beatlens)
    if stdlen > meanlen * 0.2: # irregular rhythm
        # print(‘irregular rhythm’)
        return 0, []
    beatstds = []
    for i in range(len(beats_128)):
        if np.corrcoef(avgbeat, beats_128[i])[0, 1] < 0.9:
            # print(‘corrcoef’)
            return 0, []
        else:
            beatstds.append(np.std(beats[i]))
        return np.mean(beatstds), []

def scale_data(data): # ppg scale
    rng = np.nanmax(data) - np.nanmin(data)
    minimum = np.nanmin(data)
    data = (data - minimum) / rng
    return data

In [ ]:
x = x[idx]
y = y[idx]
c = c[idx]
a = a[idx]

In [ ]:
x = x.astype(float)
y = y.astype(float)

In [ ]:
print(x.shape)
print(y.shape)
print(c.shape)

In [ ]:
case_mstds_list = []
for i in range(len(x)):
    mstd_seg, _ = process_beat(x[i].reshape(-1, 1))
    case_mstds_list.append(mstd_seg)
case_mask = list(np.where(np.array(np.squeeze(case_mstds_list)) > 0)[0])

In [ ]:
x_arr = x[case_mask]
y_arr = y[case_mask]
c_arr = c[case_mask]
import pickle
pickle.dump((x_arr, y_arr, c_arr), open('clf_processed_data.pkl', 'wb'))

In [ ]:
import pickle
x_arr, y_arr, c_arr  = pickle.load(open('clf_processed_data.pkl', 'rb'))
print(x_arr.shape)
print(y_arr.shape)
print(c_arr.shape)


In [ ]:
'''from sklearn.preprocessing import StandardScaler,MinMaxScaler

x_scaled = np.empty_like(x_arr)

Scaler = MinMaxScaler()
for i in range(x_arr.shape[0]):
    x_scaled[i,:] = Scaler.fit_transform(x_arr[i,:].reshape(-1,1)).flatten()'''

def scale_data(data): # ppg scale
    rng = np.nanmax(data) - np.nanmin(data)
    minimum = np.nanmin(data)
    data = (data - minimum) / rng
    return data

In [ ]:
#np.save('./processd_data/reg/x_ecg_reg_mms',x_ecg_scaled)
#np.save('./processd_data/reg/x_ppg_reg_mms',x_scaled)
#np.save('./processd_data/reg/x_co2_reg_mms',x_co2_scaled)

In [ ]:
import pickle

with open('clf_train_valid_test_caseids.pkl','rb') as file:
    caseids = pickle.load(file)

print(caseids)

In [ ]:
train_mask = np.isin(c_arr,caseids[0])
valid_mask = np.isin(c_arr,caseids[1])
test_mask = np.isin(c_arr,caseids[2])

print(train_mask.sum())
print(valid_mask.sum())
print(test_mask.sum())



In [ ]:
x_train = x_arr[train_mask]
y_train = y_arr[train_mask]
c_train = c_arr[train_mask]

x_valid = x_arr[valid_mask]
y_valid = y_arr[valid_mask]
c_valid = c_arr[valid_mask]

x_test = x_arr[test_mask]
y_test = y_arr[test_mask]
c_test = c_arr[test_mask]

In [ ]:
for i in range(len(x_train)):
    x_train[i,0,:] = scale_data(x_train[i,0,:]) 

for i in range(len(x_valid)):
    x_valid[i,0,:] = scale_data(x_valid[i,0,:]) 

for i in range(len(x_test)):
    x_test[i,0,:] = scale_data(x_test[i,0,:])

In [ ]:
print("caseids train cnt: {}, caseids val cnt {}, caseids test cnt: {}".format(len(caseids[0]), len(caseids[1]), len(caseids[2])))
print("samples train cnt: {}, samples val cnt: {}, samples test cnt: {}".format(len(x_train), len(x_valid), len(x_test)))

In [ ]:
print('x:',x_train.shape,'y:',y_train.shape)
print('x:',x_valid.shape,'y:',y_valid.shape)
print('x:',x_test.shape,'y:',y_test.shape)

In [ ]:

y_train = np.expand_dims(y_train,axis=-1)
y_valid = np.expand_dims(y_valid,axis=-1)
y_test = np.expand_dims(y_test,axis=-1)

print('x:',x_train.shape,'y:',y_train.shape)
print('x:',x_valid.shape,'y:',y_valid.shape)
print('x:',x_test.shape,'y:',y_test.shape)

In [ ]:
x_train_tensors = torch.tensor(x_train,dtype=torch.float32)
x_valid_tensors = torch.tensor(x_valid,dtype=torch.float32)
x_test_tensors = torch.tensor(x_test,dtype=torch.float32)

x_train_tensors = x_train_tensors.permute(0,2,1)
x_valid_tensors = x_valid_tensors.permute(0,2,1)
x_test_tensors = x_test_tensors.permute(0,2,1)

y_train_tensors = torch.tensor(y_train,dtype=torch.float32)
y_valid_tensors = torch.tensor(y_valid,dtype=torch.float32)
y_test_tensors = torch.tensor(y_test,dtype=torch.float32)

#single-channel
#X_train_tensors_f = torch.reshape(X_train_tensors,())

print("Training Shape", x_train_tensors.shape, y_train_tensors.shape)
print("Testing Shape", x_test_tensors.shape, y_test_tensors.shape) 

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

In [ ]:
train_dataset = CustomDataset(x_train_tensors,y_train_tensors)
valid_dataset = CustomDataset(x_valid_tensors,y_valid_tensors)
test_dataset = CustomDataset(x_test_tensors,y_test_tensors)

In [ ]:
train_loader = DataLoader(dataset=train_dataset,batch_size = 128, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,batch_size = 128, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size = 128, shuffle=True)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes 
        self.num_layers = num_layers 
        self.input_size = input_size 
        self.hidden_size = hidden_size 
        self.seq_length = seq_length 

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) 
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(64, num_classes)
        self.fc_2 = nn.Linear(128,64) 

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(x.device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(x.device)
        
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) 
        #rint(hn[-1].shape)
        hn = hn[-1].view(-1 , self.hidden_size)
        #print(hn.shape)

        out = self.relu(hn)
        out = self.fc_1(out) 
        out = self.relu(out)
        out = self.fc_2(out)
        out = self.relu(out) 
        out = self.fc(out)
        


        return out

In [ ]:
num_epochs = 100 
learning_rate = 1e-3 

input_size = 1 
hidden_size = 128
num_layers = 3 

num_classes = 1 
model = LSTM(num_classes, input_size, hidden_size, num_layers, x_train_tensors.shape[1]) 

criterion = torch.nn.BCELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = model.to(device) 

In [ ]:
#pytorch Early-stopping
import torch
import numpy as np 
 
 
class EarlyStopping(object):
    def __init__(self, patience=2, save_path="model.pth"):
        self._min_loss = np.inf
        self._patience = patience
        self._path = save_path
        self.__counter = 0
 
    def should_stop(self, model, loss):
        if loss < self._min_loss:
            self._min_loss = loss
            self.__counter = 0
            torch.save(model.state_dict(), self._path)
        elif loss > self._min_loss:
            self.__counter += 1
            if self.__counter >= self._patience:
                return True
        return False
   
    def load(self, model):
        model.load_state_dict(torch.load(self._path))
        return model
    
    @property
    def counter(self):
        return self.__counter

In [ ]:
class EarlyStopping_score(object):
    """score로 stopping하기"""
    def __init__(self, patience, save_path, eps):
        self._max_score = -1
        self._patience = patience
        self._path = save_path
        self._eps = eps
        self.__counter = 0
 
    def should_stop(self, model, score):
        if score > self._max_score:
            self._max_score = score
            self.__counter = 0
            torch.save(model.state_dict(), self._path)
        elif score < self._max_score + self._eps:
            self.__counter += 1
            if self.__counter >= self._patience:
                return True
        return False

In [ ]:
from tqdm import tqdm
import numpy as np
import torch
from sklearn.metrics import roc_auc_score,precision_score, recall_score, average_precision_score

early_stopper = EarlyStopping_score(patience=3,save_path='./clf/best_model_clf.pth',eps=0.001)

loss_history = {'train':[],
                'valid':[],
                'test':[]}
acc_history = {'train':[],
                'valid':[],
                'test':[]}
recall_history = {'train':[],
                'valid':[],
                'test':[]}

model = model.to(device)
for epoch in range(num_epochs):
    train_loss = 0
    train_corrects = 0
    model.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        for x_train, y_train in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            x_train, y_train = x_train.to(device), y_train.to(device) 
            optimizer.zero_grad()
            outputs = model(x_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer.step()
            tepoch.set_postfix(loss=loss.item())
            
            preds = torch.round(outputs) #binaryclassification
            train_loss += loss.item()
            train_corrects += torch.sum(preds == y_train)
        
        train_loss = train_loss / len(train_loader)
        train_acc = train_corrects.double() / len(train_loader.dataset)
        

    all_probs = []
    all_labels = []
    all_preds = []
    

    # 검증
    model.eval()
    with torch.no_grad():
        valid_loss = 0
        valid_corrects = 0
        for i, batch in enumerate(tqdm(valid_loader)):
            x_valid, y_valid = batch
            x_valid, y_valid = x_valid.to(device), y_valid.to(device) 
            
            outputs = model(x_valid)
            loss = criterion(outputs, y_valid)
            preds = torch.round(outputs) #binaryclassification
            valid_loss += loss.item()
            valid_corrects += torch.sum(preds == y_valid)
           
            all_probs.extend(outputs.cpu().numpy())
            all_labels.extend(y_valid.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
        
        valid_loss = valid_loss / len(valid_loader)
        valid_acc = valid_corrects.double() / len(valid_loader.dataset)
        valid_recall = recall_score(all_labels,all_preds)
        valid_precision = precision_score(all_labels,all_preds)

    all_probs_array = np.array(all_probs)
    auc_score = roc_auc_score(all_labels, all_probs_array)
    auprc = average_precision_score(all_labels, all_probs)

    loss_history['train'].append(train_loss)
    loss_history['valid'].append(valid_loss)

    acc_history['train'].append(train_acc)
    acc_history['valid'].append(valid_acc)
    recall_history['valid'].append(valid_recall)

    if early_stopper.should_stop(model,auc_score):
        print(f'Early Stopping')
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss:{train_loss:.3f} | Train acc:{train_acc:.3f}')
        print(f'Valid Loss:{valid_loss:.3f} | Valid acc:{valid_acc:.3f} | Valid_auc:{auc_score:.3f}')
        print(f'Valid recall:{valid_recall:.3f} | Valid precision: {valid_precision:.3f} | Valid AUPRC: {auprc:.3f}')
        #torch.save(model.state_dict(),'./clf/best_model_clf.pth')
        break
    
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Train Loss:{train_loss:.3f} | Train acc:{train_acc:.3f}')
    print(f'Valid Loss:{valid_loss:.3f} | Valid acc:{valid_acc:.3f} | Valid_auc:{auc_score:.3f}')
    print(f'Valid recall:{valid_recall:.3f} | Valid precision: {valid_precision:.3f} | Valid AUPRC: {auprc:.3f}')




test_loss = 0
test_corrects = 0
test_probs = []
test_labels = []
test_preds = []

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        x_test, y_test = batch
        x_test, y_test = x_test.to(device), y_test.to(device)

        outputs = model(x_test)
            
        loss = criterion(outputs, y_test)
        preds = torch.round(outputs) #binaryclassification
        test_corrects += torch.sum(preds == y_test)
        test_loss += loss.item()
        test_probs.extend(outputs.cpu().numpy())
        test_labels.extend(y_test.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())
            
    test_loss = test_loss / len(test_loader)
    test_acc = test_corrects.double() / len(test_loader.dataset)
    test_recall = recall_score(test_labels,test_preds)
    test_precision = precision_score(test_labels,test_preds)
        
    loss_history['test'].append(test_loss)
    acc_history['test'].append(test_acc)
    recall_history['test'].append(test_recall)

    test_probs_array = np.array(test_probs)
    auc_score_test = roc_auc_score(test_labels, test_probs_array)
    auprc_test = average_precision_score(test_labels, test_probs_array)
    


print(f'Test Loss:{test_loss:.3f} | Test acc:{test_acc:.3f} | Test_auc:{auc_score_test:.3f}')
print(f'Test Recall:{test_recall:.3f} | Test Precison:{test_precision:.3f} | Test_auprc:{auprc_test:.3f}')
